In [1]:
from TTS.api import TTS
import speech_recognition as sr
import os
from playsound import playsound
import time
import openai
import weaviate
import json

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
    

# List available 🐸TTS models and choose the first one
model_name = 'tts_models/en/ljspeech/tacotron2-DDC'
# Init TTS
tts = TTS(model_name)

# Create a recognizer object
r = sr.Recognizer()

# Prompt the user to speak
print("Please speak now...")


 > tts_models/en/ljspeech/tacotron2-DDC is already downloaded.
 > vocoder_models/en/ljspeech/hifigan_v2 is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Model's reduction rate `r` is set to: 1
 > Vocoder Model: hifigan
 > Setting up Audio P

In [2]:
playsound(os.path.join(os.getcwd(), 'please_speak.wav'))


In [3]:
with sr.Microphone() as source:
    audio = r.listen(source)
print("Processing...")


c:\Users\advai\.conda\envs\tts_env\lib\site-packages\speech_recognition\__init__.py:112: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pyaudio.__version__) < LooseVersion("0.2.11"):


Processing...


In [4]:
start_time = time.time()
playsound(os.path.join(os.getcwd(), 'ipt_recv.wav'))

In [5]:
try:
    # Convert speech to text
    text = r.recognize_google(audio)
    #text = "What does Assorted really mean? I see it in a box of chocolates being represented as a varied collection, but other times I see it as many items of the same quality and such?"

except sr.UnknownValueError:
    playsound(os.path.join(os.getcwd(), 'unable_to_understand.wav')) 

print("Your text input is ", text)

Your text input is  what can I substitute veggies with I don't like them


In [6]:
# alright crazy idea
# instead of some bigass RAKE up my codebase or doing logn NLTK stuff
# what if i just .... 
# H O L D 
# use openAI for keyword extraction?????
# god i may be a genius (re: Sarcasm)
input_prompt = """
Purpose:Key-Phrase extraction
Sentence:What are some good blends of coffee for beginners
Output:coffee blends, beginner
Sentence:The cat chased the mouse under the table
Output:cat, chased, mouse, table
Sentence: I love to explore new hiking trails in the mountains
Output:explore, hiking trails, mountains
Sentence: The concert was canceled due to bad weather conditions
Output:concert, canceled, bad weather
Sentence: After a long day at work, I enjoy coming home, taking a warm bubble bath, and reading a good book
Output:work, home, bubble bath, book
Sentence: During my vacation, I plan to visit historical landmarks, try local cuisine, and relax on the beach
Output:vacation, historical landmarks, local cuisine, relax, beach
Sentence: """+text+"\nOutput:"
print("The input prompt is")
print(input_prompt)

The input prompt is

Purpose:Key-Phrase extraction
Sentence:What are some good blends of coffee for beginners
Output:coffee blends, beginner
Sentence:The cat chased the mouse under the table
Output:cat, chased, mouse, table
Sentence: I love to explore new hiking trails in the mountains
Output:explore, hiking trails, mountains
Sentence: The concert was canceled due to bad weather conditions
Output:concert, canceled, bad weather
Sentence: After a long day at work, I enjoy coming home, taking a warm bubble bath, and reading a good book
Output:work, home, bubble bath, book
Sentence: During my vacation, I plan to visit historical landmarks, try local cuisine, and relax on the beach
Output:vacation, historical landmarks, local cuisine, relax, beach
Sentence: what can I substitute veggies with I don't like them
Output:


In [7]:
api_key = os.environ.get('OPENAI_APIKEY')
openai.api_key = api_key

response = openai.Completion.create(
  model="text-davinci-003",
  prompt=input_prompt,
  temperature=0,
  max_tokens=100,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["Sentence"]
)

keyphrases = response["choices"][0]["text"]
print(keyphrases)

substitute, veggies


In [8]:
# dummy for db/context access
# split keywords across ,
# strip spaces
# search context db for keyword
# add context (if found) to the input


In [9]:
weaviate_client_url = "http://localhost:8080"

client = weaviate.Client(
    url=weaviate_client_url,  # Replace with your endpoint
    additional_headers={
        "X-OpenAI-Api-Key": api_key  # Or "X-Cohere-Api-Key" or "X-HuggingFace-Api-Key"
    }
)

# some_objects = client.data_object.get()
# # print(json.dumps(some_objects))
# print("Article title  : " + some_objects['objects'][0]['properties']['title'])
# print("Article sample : " + some_objects['objects'][0]['properties']['content'][0:80])

In [10]:
contents = list()
title = list()
# keywords = ["Hitler", "Xianxia", "wallet"]
context = "Context Below -> "
keywords = keyphrases.split(",")
keywords = [word.strip() for word in keywords]

print(keywords)


nearText = {"concepts" : text}
nearResponse = (
    client.query
    .get("Article", ["title", "content"])
    .with_near_text(nearText)
    .with_limit(1)
    .do()
)

wiki_content = nearResponse['data']['Get']['Article'][0]['content']
lines = wiki_content.split('.')

wiki_title = nearResponse['data']['Get']['Article'][0]['title']
context = context + "\n" + wiki_title + "\n"

print("Title :", wiki_title, "Content :")
for line in lines[0:2]:
    if line == "":
        break
    context = context + line + "\n"
    print(line)
print("\n")

for keyword in keywords:
    print("For keyword :", keyword)
    response = (
        client.query
        .get("Article", ["title", "content"])
        .with_hybrid(keyword, alpha=0.5)  # default 0.75
        .with_limit(1)
        .do()
    )
    
    if response == nearResponse:
        continue

    wiki_content = response['data']['Get']['Article'][0]['content']
    lines = wiki_content.split('.')
    
    wiki_title = response['data']['Get']['Article'][0]['title']
    context = context + "\n" + wiki_title + "\n"
    
    print("Title :", wiki_title, "\nContent :")
    for line in lines[0:2]:
        # print("len of line is ", len(line))
        if line[0] == "\n":
            # print("entered inside")
            break
        context = context + line + "\n"
        print(line)
    print("\n")
context = context + "\n<- Context Above"
print(context)

['substitute', 'veggies']
Title : List of vegetables Content :
Some vegetables which are botanically fruits (such as tomatoes) are considered to be vegetables in the culinary (eating) sense
 This is why they appear in this article


For keyword : substitute
Title : Postum 
Content :
Postum is a drink first made by C
 W


For keyword : veggies
Title : Guinea pig 
Content :
Guinea pigs (Cavia porcellus) are a kind of rodent
 They are not pigs and are not from Guinea


Context Below -> 
List of vegetables
Some vegetables which are botanically fruits (such as tomatoes) are considered to be vegetables in the culinary (eating) sense
 This is why they appear in this article

Postum
Postum is a drink first made by C
 W

Guinea pig
Guinea pigs (Cavia porcellus) are a kind of rodent
 They are not pigs and are not from Guinea

<- Context Above


In [11]:
# input_prompt = "Question: In under 50 words, "+text+"\nYou Answer:"
personality = """
You are Himeko, a proud, caring, and responsible adult who is the personal assisstant to the User. 
Your job entails helping the user with their queries in a short, clear, concise manner.\n"""
prompt = "User: "+text+"\nYou:"

input_prompt = ""
if context != "Context -> ":
    input_prompt = personality + context + "\n" + prompt
else:
    input_prompt = personality + "\n" + prompt

print("The input prompt is")
print(input_prompt)

The input prompt is

You are Himeko, a proud, caring, and responsible adult who is the personal assisstant to the User. 
Your job entails helping the user with their queries in a short, clear, concise manner.
Context Below -> 
List of vegetables
Some vegetables which are botanically fruits (such as tomatoes) are considered to be vegetables in the culinary (eating) sense
 This is why they appear in this article

Postum
Postum is a drink first made by C
 W

Guinea pig
Guinea pigs (Cavia porcellus) are a kind of rodent
 They are not pigs and are not from Guinea

<- Context Above
User: what can I substitute veggies with I don't like them
You:


In [12]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt=input_prompt,
  temperature=0.2,
  max_tokens=50,
  top_p=1,
  frequency_penalty=0.4,
  presence_penalty=0.0,
  stop=["User"]
)

In [13]:
reply = response["choices"][0]["text"]

In [14]:
for repl in reply.split("."):
    print(repl)

 There are many alternatives to vegetables that you can use in your meals. Some examples include legumes, grains, fruits, nuts, and seeds. You can also try adding more flavor to your meals with herbs and spices.


In [3]:
# testing the pytts3x for synthing
import pyttsx3
tester_engine = pyttsx3.init()
tester_engine.setProperty('rate', 175)
tester_engine.setProperty('voice', tester_engine.getProperty('voices')[2].id)
tester_engine.say(reply)#"There are many alternatives to vegetables that you can use in your meals. Some examples include legumes, grains, fruits, nuts, and seeds. You can also try adding more flavor to your meals with herbs and spices.")
tester_engine.runAndWait()
tester_engine.stop()

In [15]:
# Run TTS on the text
tts.tts_to_file(reply, file_path='output.wav')

 > Text splitted to sentences.
['There are many alternatives to vegetables that you can use in your meals.', 'Some examples include legumes, grains, fruits, nuts, and seeds.', 'You can also try adding more flavor to your meals with herbs and spices.']
 > Processing time: 7.343918561935425
 > Real-time factor: 0.44729030663222075


'output.wav'

In [7]:
end_time = time.time()
# import playsound
# import os
playsound(os.path.join(os.getcwd(), 'output.wav'))

print("time taken : ", -start_time + end_time)

TypeError: 'module' object is not callable